## 1) Setup + configuración

In [ ]:
from dotenv import load_dotenv
import os
from pathlib import Path


load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
assert OPENAI_API_KEY, "Missing OPENAI_API_KEY at .env"

DATA_DIR = Path("data")
assert DATA_DIR.exists(), "data/ folder does not exist!"

print("✅ Environment OK")

In [ ]:
BOE_PDF = DATA_DIR / "BOE-A-2020-8608.pdf"
SENTINEL_PDF = DATA_DIR / "sentinel_secure_services_maual_operativo.pdf"  # noqa

print("BOE exists:", BOE_PDF.exists())
print("Sentinel exists:", SENTINEL_PDF.exists())

## 2) Carga + extracción

In [37]:
from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader


def load_pdf(path: Path) -> list[Document]:
    loader = PyPDFLoader(str(path))
    documents = loader.load()
    return documents


docs_boe = load_pdf(BOE_PDF) if BOE_PDF.exists() else []
docs_sentinel = load_pdf(SENTINEL_PDF) if SENTINEL_PDF.exists() else []

docs = docs_boe
print(f"Pages loaded: {len(docs)}")
print(f"Example metadata: {docs[0].metadata}" if docs else None)
print(f"Sample text: {docs[0].page_content[:400]} [...]" if docs else "No docs")

Pages loaded: 42
Example metadata: {'producer': 'Antenna House PDF Output Library 6.5.1216 (Linux64)', 'creator': 'eBOE', 'creationdate': '2020-07-25T02:03:33+01:00', 'keywords': 'ORDEN TMA/702/2020 de 15/07/2020;"MINISTERIO DE TRANSPORTES, MOVILIDAD Y AGENDA URBANA";BOE-A-2020-8608;BOE 203 de 2020;8608;27/07/2020', 'moddate': '2020-07-25T02:15:13+02:00', 'trapped': '/False', 'subject': 'BOE-A-2020-8608', 'author': 'MINISTERIO DE TRANSPORTES, MOVILIDAD Y AGENDA URBANA', 'title': 'Disposición 8608 del BOE núm. 203 de 2020', 'source': 'data/BOE-A-2020-8608.pdf', 'total_pages': 42, 'page': 0, 'page_label': '1'}
Sample text: III. OTRAS DISPOSICIONES
MINISTERIO DE TRANSPORTES, MOVILIDAD
Y AGENDA URBANA
8608 Orden TMA/702/2020, de 15 de julio, por la que se aprueban las bases 
reguladoras para la concesión por Puertos del Estado de ayudas públicas en 
el marco del Plan de Impulso al Emprendimiento para la Innovación en el 
Sector Portuario («Puertos 4.0») y se convocan las ayudas para 2020.


In [38]:
def doc_stats(documents: list[Document]) -> tuple[int, int]:
    total_characters = sum(len(d.page_content) for d in documents)
    average_characters = total_characters / max(1, len(documents))
    return total_characters, average_characters


total_chars, avg_chars = doc_stats(docs)
print(f"Total characters: {total_chars:_}\nAvg characters per page: {avg_chars:_.0f}")

Total characters: 135_819
Avg characters per page: 3_234


## 3) Chunking

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


def split_docs(documents: list[Document], chunk_size: int = 800, chunk_overlap: int = 120) -> list[Document]:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ". ", " ", ""],
    )
    return splitter.split_documents(documents)


chunks_200 = split_docs(docs, chunk_size=200, chunk_overlap=50)
chunks_800 = split_docs(docs, chunk_size=800, chunk_overlap=120)
chunks_2000 = split_docs(docs, chunk_size=2000, chunk_overlap=200)

print(f"Chunks (200): {len(chunks_200)}")
print(f"Chunks (800): {len(chunks_800)}")
print(f"Chunks (2000): {len(chunks_2000)}")

In [39]:
import textwrap


def preview_chunks(chunks: list[Document], n: int = 2, width: int = 100, max_chars: int = 800) -> None:
    for i in range(min(n, len(chunks))):
        md = chunks[i].metadata
        txt = chunks[i].page_content.strip().replace("\n", " ")
        print(f"\n—Chunk {i} | source={md.get('source')}; page={md.get('page')}; len={len(chunks[i].page_content):_}")
        print(textwrap.fill(txt[:max_chars], width=width))


preview_chunks(chunks_2000, n=1, max_chars=800)
preview_chunks(chunks_200, n=1)


—Chunk 0 | source=data/BOE-A-2020-8608.pdf; page=0; len=1_994
III. OTRAS DISPOSICIONES MINISTERIO DE TRANSPORTES, MOVILIDAD Y AGENDA URBANA 8608 Orden
TMA/702/2020, de 15 de julio, por la que se aprueban las bases  reguladoras para la concesión por
Puertos del Estado de ayudas públicas en  el marco del Plan de Impulso al Emprendimiento para la
Innovación en el  Sector Portuario («Puertos 4.0») y se convocan las ayudas para 2020. En el
contexto de la 4.ª revolución industrial, en la que se encuentran todos los  sectores económicos,
éstos están siendo sometidos a procesos de transformación  disruptiva creando productos, servicios y
procesos innovadores gracias a la utilización de  las últimas tecnologías. Como no podía ser de otra
forma, esta necesidad de  transformación también se da en el sector del comercio, la logística, el
transporte, y  engloba a

—Chunk 0 | source=data/BOE-A-2020-8608.pdf; page=0; len=151
III. OTRAS DISPOSICIONES MINISTERIO DE TRANSPORTES, MOVILIDAD Y AGENDA URBA

## 4) Embeddings + vector store

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma


EMBED_MODEL = "text-embedding-3-small"


def build_chroma(chunks: list[Document], persist_dir="chroma_boe", collection="boe") -> Chroma:
    embeddings = OpenAIEmbeddings(model=EMBED_MODEL)
    db = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory=persist_dir,
        collection_name=collection,
    )
    return db


db_800 = build_chroma(chunks_800, persist_dir="chroma_boe_800", collection="boe_800")
print("✅ Chroma built (800).")

In [33]:
def retrieve_with_scores(db: Chroma, query_: str, n_results: int = 4) -> list[tuple[Document, float]]:
    return db.similarity_search_with_score(query_, k=n_results)


def print_chroma_hits(hits: list[tuple[Document, float]], max_chars: int = 600, width: int = 110) -> None:
    print(f"Score: less is better for cosine-distance based search.")
    for i, (doc, score) in enumerate(hits):
        md = doc.metadata
        print(f"\n#{i} score={score:.4f}; page={md.get('page')}; source={md.get('source')}")
        print(textwrap.fill(doc.page_content[:max_chars].replace("\n", " "), width=width))


query = "¿Cuál es el porcentaje máximo de subvención para proyectos en fase comercial?"  # noqa
results = retrieve_with_scores(db_800, query, n_results=4)
print_chroma_hits(results)

Score: less is better for cosine-distance based search.

#0 score=0.5126; page=9; source=data/BOE-A-2020-8608.pdf
Proyectos, tanto en fase comercial como en fase pre-comercial, la subcontratación podrá  alcanzar hasta el
porcentaje máximo del 70 % del presupuesto financiable. CAPÍTULO IV Régimen de las ayudas Artículo 14.
Modalidades de ayuda. 1. Las aportaciones se harán en concepto de subvención y se otorgarán en  régimen de
concurrencia competitiva, conforme al régimen establecido en los  artículos 3.2 y 22.1 de la Ley 38/2003. 2.
Las ayudas consisten en una aportación dineraria para el desarrollo de la idea o  del proyecto seleccionado y
sujetarán al beneficiario al cumplimiento de las obligaciones  B

#1 score=0.6169; page=22; source=data/BOE-A-2020-8608.pdf
finalización del proyecto subvencionado. Para proyectos en fase pre-comercial se  asignarán 20 puntos si el
compromiso de  lanzamiento al mercado se produce en el  periodo máximo de dos años desde la  finalización del
proyecto

In [ ]:
db_2000 = build_chroma(chunks_2000, persist_dir="chroma_boe_2000", collection="boe_2000")
db_200 = build_chroma(chunks_200, persist_dir="chroma_boe_200", collection="boe_200")

print("\n\nUsing chunk size 2000:")
results = retrieve_with_scores(db_2000, query, n_results=3)
print_chroma_hits(results)

print("\n\nUsing chunk size 200:")
results = retrieve_with_scores(db_200, query, n_results=3)
print_chroma_hits(results)

## 6) Generación

In [32]:
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.0)


def answer_no_context(question_: str) -> str:
    return llm.invoke(question_).content


def answer_with_context(question_: str, retrieved_docs: list[Document]) -> str:
    context = "\n\n".join(
        f"[source={d.metadata.get('source')}; page={d.metadata.get('page')}]\n{d.page_content}"
        for d in retrieved_docs
    )

    # noinspection SpellCheckingInspection
    prompt = f"""Usa el siguiente contexto para responder la pregunta.
Si la respuesta no está contenida en el contexto, di "no lo sé".

Contexto:
{context}

Pregunta:
{question_}
"""
    return llm.invoke(prompt).content

In [36]:
question = "¿Cuál es el porcentaje máximo de subvención para proyectos en fase comercial?"  # noqa

print("=== No context ===")
print(answer_no_context(question))

print("\n=== With RAG context (chunk size 800) ===")
retrieved = [doc for doc, _score in retrieve_with_scores(db_800, question, n_results=4)]
print(answer_with_context(question, retrieved))

=== No context ===
El porcentaje máximo de subvención para proyectos en fase comercial puede variar dependiendo del programa, la entidad financiadora y el país. Sin embargo, en muchos programas de apoyo a la innovación y desarrollo tecnológico, la subvención para proyectos en fase comercial suele ser menor que para fases más tempranas (como investigación o desarrollo experimental), debido a que el riesgo es menor y se espera que el proyecto ya tenga cierta viabilidad comercial.

Por ejemplo, en programas de la Unión Europea como Horizonte Europa, o en programas nacionales de innovación, el porcentaje máximo de subvención para proyectos en fase comercial suele estar alrededor del **25% al 40%** del coste elegible, aunque esto puede variar.

Para darte una respuesta precisa, necesitaría saber a qué programa o convocatoria te refieres específicamente. ¿Podrías proporcionarme más detalles?

=== With RAG context (chunk size 800) ===
El porcentaje máximo de subvención para proyectos en fase 

In [42]:
question = "¿Cuál es el porcentaje máximo de subvención para proyectos en fase comercial?"  # noqa

print("\n=== With RAG context (chunk size 200) ===")
retrieved = [doc for doc, _score in retrieve_with_scores(db_200, question, n_results=4)]
print(answer_with_context(question, retrieved))

print("\n=== With RAG context (chunk size 800) ===")
retrieved = [doc for doc, _score in retrieve_with_scores(db_800, question, n_results=4)]
print(answer_with_context(question, retrieved))

print("\n=== With RAG context (chunk size 2_000) ===")
retrieved = [doc for doc, _score in retrieve_with_scores(db_2000, question, n_results=4)]
print(answer_with_context(question, retrieved))


=== With RAG context (chunk size 200) ===
No lo sé.

=== With RAG context (chunk size 800) ===
El porcentaje máximo de subvención para proyectos en fase comercial, en cuanto a subcontratación, podrá alcanzar hasta el 70 % del presupuesto financiable.

=== With RAG context (chunk size 2_000) ===
No lo sé.
